## Data munging starts: 
### Importing packages and data

In [67]:
import datetime
import numpy as np
from numpy import double
import pandas as pd

In [68]:
print("Loading data start...")
# train and test set provided by Rossmann
train = pd.read_csv("input/train.csv")
test = pd.read_csv("input/test.csv")
print("Complete!")

Loading data start...
Complete!


In [69]:
print("Loading data start...")
# external data
store = pd.read_csv("input/store.csv")
states = pd.read_csv("input/store_states.csv")
eco_data = pd.read_csv("input/economic_data.csv")
print("Complete!")

Loading data start...
Complete!


## Data munging part I:
### Clean up, merges and dates features

In [70]:
# 1: Impute Open = 1 for store 622 in test data
test['Open'].fillna(1, inplace=True)
print 'Step 1 Complete.'

Step 1 Complete.


In [71]:
# 2: Merging the train and test for easy implementation of features
train['type'] = 'train'
test['type'] = 'test'
all_data = [train, test]
all_data = pd.concat(all_data)
print 'Step 2 Complete'

Step 2 Complete


In [72]:
# 3: Separate date columns
all_data['year']  = all_data.Date.apply(lambda x: x.split('-')[0])
all_data['year']  = all_data['year'].astype(int)
all_data['month'] = all_data.Date.apply(lambda x: x.split('-')[1])
all_data['month'] = all_data['month'].astype(int)
all_data['day']   = all_data.Date.apply(lambda x: x.split('-')[2])
all_data['day']   = all_data['day'].astype(int)
print 'Step 3 Complete.'

Step 3 Complete.


In [73]:
# 4: Withdraw multiple date features
all_data['Date']             = pd.to_datetime(all_data['Date'])
all_data['day_of_year']      = all_data['Date'].dt.dayofyear
all_data['quarter']          = all_data['Date'].dt.quarter
all_data['is_month_start']   = all_data['Date'].dt.is_month_start.astype(int)
all_data['is_month_end']     = all_data['Date'].dt.is_month_end.astype(int)
all_data['is_quarter_start'] = all_data['Date'].dt.is_quarter_start.astype(int)
all_data['is_quarter_end']   = all_data['Date'].dt.is_quarter_end.astype(int)
all_data['weeknum']          = np.ceil(all_data['day_of_year']/7.)
print 'Step 4 Complete.'

Step 4 Complete.


In [74]:
# 5: Standardize StateHoliday column.
all_data.ix[(all_data['StateHoliday'] == 0), 'StateHoliday'] = '0'
print 'Step 5 Complete.'

Step 5 Complete.


In [75]:
# 6: Setting 'Open' to 0 if Sales are 0
all_data.ix[(all_data['Sales'] == 0), 'Open'] = 0
print 'Step 6 Complete.'

Step 6 Complete.


In [76]:
# 7: Create duration of promo 1
all_data = all_data.sort(columns = ['Store', 'Date'], ascending = True, na_position = 'last')
all_data['PromoFirstDate'] = 0
all_data.ix[((all_data['Store'] == all_data['Store'].shift(1)) &
          (all_data['Promo'] == 1 ) & (all_data['Promo'].shift(1) ==0)),
         'PromoFirstDate'] = 1
print 'Step 7 Complete.'

Step 7 Complete.


In [77]:
# 8: Joining States:
all_data = pd.merge(all_data, states, on = 'Store', how = 'inner')
print 'Step 8 Complete.'

Step 8 Complete.


## Data munging part II: 
### Adding some usefull functions:

In [78]:
# f1: Creating a function to summarize our data
def rstr(df): 
    print df.shape
    print '=' * 72
    print df.apply(lambda x: [x.unique()])
    print '=' * 72
    print pd.isnull(df).any()
print 'function 1: rstr added'

function 1: rstr added


In [79]:
# f2: Creating a function to shift columns and making sure to distinguish between stores
def shift_col(df, col_name, n):
    df[col_name + str(n)] = df[col_name].shift(n)
    df.ix[(df['Store'] != df['Store'].shift(n)), col_name + str(n)] = float('NaN')
print 'function 2: shif_col added'    

function 2: shif_col added


In [80]:
# f2.1: Demonstrating what shift_col does:
# temp = pd.DataFrame({
#         'Sales': [1,2,3,4,5,1,2,3,4,5],
#         'Store': [1,1,1,1,1,2,2,2,2,2]
#     })
# shift_col(temp, 'Sales', 2)
# shift_col(temp, 'Sales', -1)
# temp

## Data munging part III: 
### Adding Schedule and historic Sales

In [81]:
# 9: now we can add previous sales for the past 21 days:
for i in range(1,22):
    shift_col(all_data, 'Sales', i)
print 'Step 9 Complete.'

Step 9 Complete.


In [82]:
# 10: now we can add previous open for the past 21 days:
for i in range(1,22):
    shift_col(all_data, 'Open', i)
print 'Step 10 Complete.'

Step 10 Complete.


In [83]:
# 11: now we can add next open for the past 21 days:
for i in range(1,22):
    shift_col(all_data, 'Open', -i)
print 'Step 11 Complete.'

Step 11 Complete.


In [84]:
all_data.head(1)

,Customers,Date,DayOfWeek,Id,Open,Promo,Sales,SchoolHoliday,StateHoliday,Store,...,Open-12,Open-13,Open-14,Open-15,Open-16,Open-17,Open-18,Open-19,Open-20,Open-21
0,0,2013-01-01,2,NaN,0,0,0,1,a,1,...,0,1,1,1,1,1,1,0,1,1


## Data munging part IV: 
### Adding all weather data with historic and forecast

In [85]:
# 12: loading all weather data into one dataFrame called weather:
weather = pd.DataFrame()
for i in all_data['State'].unique():
    i = 'NI' if i == 'HB,NI' else i
    temp = pd.read_csv("data_for_features/Data_Weather/" + str(i) + ".csv", sep = ";")

    i = 'HB,NI' if i == 'NI' else i
    temp['State'] = i
    
    temp = [weather, temp]
    weather = pd.concat(temp)

weather['Date'] = pd.to_datetime(weather['Date'])
print 'Step 12 Complete.'

Step 12 Complete.


In [86]:
# 12.1: Checking that step 12 was ok especially for state 'HB,NI'
# weather[weather['State']=='HB,NI']

In [87]:
weather.head(1)

,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,Min_Humidity,...,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees,State
0,2013-01-01,8,6,3,6,3,1,93,80,59,...,12,10,23,14,39,2.03,6,Rain,206,HE


In [88]:
# 13: Merging the weather data to our dataset:
all_data = pd.merge(all_data, weather, on = ['State', 'Date'], how = 'left')
print 'Step 13 Complete.'

Step 13 Complete.


In [99]:
all_data.head(1)

,Customers,Date,DayOfWeek,Id,Open,Promo,Sales,SchoolHoliday,StateHoliday,Store,...,IT_NET_USER_P2,TG_VAL_TOTL_GD_ZS,BX_TRF_PWKR_CD_DT,BX_KLT_DINV_CD_WD,HHDEBT,HHFA,HHDI,HHFT,HHSAVFORECAST,HHSAV
0,0,2013-01-01,2,NaN,0,0,0,1,a,1,...,82.349998,72.69033,1.643390e+10,5.465988e+10,95.47638,39.5527,0.623393,8.310776,9.13566,9.260378


In [89]:
# 14: Adding some weather historic on mean temperature, max wind speed, precipitation and events:
# last 14 days
for i in range(1,15):
    shift_col(all_data, 'Mean_TemperatureC', i)

# next 3 days (forecast)
for i in range(1,4):
    shift_col(all_data, 'Mean_TemperatureC', -i)

print 'Step 14 Complete.'

Step 14 Complete.


In [90]:
# 15: Adding some weather historic on mean temperature, max wind speed, precipitation and events:
# last 14 days
for i in range(1,15):
    shift_col(all_data, 'Max_Wind_SpeedKm_h', i)

# next 3 days (forecast)
for i in range(1,4):
    shift_col(all_data, 'Max_Wind_SpeedKm_h', -i)

print 'Step 15 Complete.'

Step 15 Complete.


In [91]:
# 16: Adding some weather historic on mean temperature, max wind speed, precipitation and events:
# last 14 days
for i in range(1,15):
    shift_col(all_data, 'Precipitationmm', i)

# next 3 days (forecast)
for i in range(1,4):
    shift_col(all_data, 'Precipitationmm', -i)

print 'Step 16 Complete.'

Step 16 Complete.


In [92]:
# 17: Adding some weather historic on mean temperature, max wind speed, precipitation and events:
# last 14 days
for i in range(1,15):
    shift_col(all_data, 'Events', i)

# next 3 days (forecast)
for i in range(1,4):
    shift_col(all_data, 'Events', -i)

print 'Step 17 Complete.'

Step 17 Complete.


In [93]:
all_data.head(1)

,Customers,Date,DayOfWeek,Id,Open,Promo,Sales,SchoolHoliday,StateHoliday,Store,...,Events8,Events9,Events10,Events11,Events12,Events13,Events14,Events-1,Events-2,Events-3
0,0,2013-01-01,2,NaN,0,0,0,1,a,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rain,Rain


## Data munging part V: 
### Adding all economic data

In [94]:
# 18: coverting date to match our all_data 'Date' otherwise merging crashes
eco_data['Date'] = pd.to_datetime(eco_data['Date'])
print 'Step 18 Complete'

Step 18 Complete


In [95]:
eco_data.head(1)

,Date,DAX,Merck,MSCI,CLeadIndic,BusConf,ConsConf,SP_POP_TOTL,SP_POP_GROW,EN_POP_DNST,...,IT_NET_USER_P2,TG_VAL_TOTL_GD_ZS,BX_TRF_PWKR_CD_DT,BX_KLT_DINV_CD_WD,HHDEBT,HHFA,HHDI,HHFT,HHSAVFORECAST,HHSAV
0,2013-01-01,7612.390137,99.83,24.4,99.22213,99.83537,99.86818,80425823,-1.691349,230.750625,...,82.349998,72.69033,1.643390e+10,5.465988e+10,95.47638,39.5527,0.623393,8.310776,9.13566,9.260378


In [96]:
# 19: merging all eco data to our data
all_data = pd.merge(all_data, eco_data, on = ['Date'], how = 'left')
print 'Step 19 Complete'

Step 19 Complete


In [97]:
all_data.head(1)

,Customers,Date,DayOfWeek,Id,Open,Promo,Sales,SchoolHoliday,StateHoliday,Store,...,IT_NET_USER_P2,TG_VAL_TOTL_GD_ZS,BX_TRF_PWKR_CD_DT,BX_KLT_DINV_CD_WD,HHDEBT,HHFA,HHDI,HHFT,HHSAVFORECAST,HHSAV
0,0,2013-01-01,2,NaN,0,0,0,1,a,1,...,82.349998,72.69033,1.643390e+10,5.465988e+10,95.47638,39.5527,0.623393,8.310776,9.13566,9.260378


## Saving our files:

In [98]:
# Create a folder 'data' right outside the folder 'KaggleProject'
all_data[all_data['type']=='train'].to_csv("../data/train.csv", index=False)
all_data[all_data['type']=='test'].to_csv("../data/test.csv", index=False)
print 'Files are saved and ready'

Files are saved and ready
